Skolemization-Related Explorations
==================

This is a temporary notebook for exploring issues related to a skolemize() method for KnownTruth objects, in particular for known truths involving universal ($\forall$) and existential ($\exists$) quantifiers.

The notebook was generated initially as a copy of the Forall tutorial notebook, and thus while under construction many of the cells and other elements of the original notebook remain.

($\forall$) Universal quantification is another core concept in **Prove-It**.  A `Forall` operation, formatted with the $\forall$ symbol, is used to represent universal quantification.  For example, $\forall_x P(x)$ means that $P(x)$ is true for any instance of $x$.  $P(x)$ holds true universally over instances of $x$.  Like `Implies`, `Forall` is a core concept but is defined outside of the core in the `proveit.logic` package. It is known in the core for use in the *specialization* and *generalization* derivation steps discussed below.

First, let us import some necessary information then consider an example of a `Forall` object like $\forall_{x \,\in\, S \,|\, Q(x), R(x)} P(x)$:

In [1]:
from proveit import Function, ExprList, Lambda, Literal
from proveit.logic import Forall, Exists, InSet, Equals
from proveit.logic.equality._axioms_ import substitution
from proveit.number import Less, Add, Frac, zero
from proveit._common_ import a, b, f, x, y, z, fx, P, Px, Pxy, Q, Qx, R, Rx, Ry, S, T
%begin skolemization_explorations

In [2]:
basicForallExpr = Forall(x, Px, conditions=[Qx, Rx], domain=S)

basicForallExpr: forall_{x in S | Q(x) , R(x)} P(x)

In [3]:
Qa = Function(Q, a)
Ra = Function(R, a)
Lc = Literal('c')
basicForallExpr.specialize({x:a}, assumptions=(basicForallExpr, InSet(a, S), Qa, Ra))

{forall_{x in S | Q(x) , R(x)} P(x) , a in S , Q(a) , R(a)} |- P(a)

In [4]:
Qa = Function(Q, a)
Ra = Function(R, a)
Rb = Function(R, b)
Lc = Literal('c')
Ld = Literal('d')
QLc = Function(Q, Lc)
RLc = Function(R, Lc)
basicForallExprSpec = basicForallExpr.specialize({x:Lc}, assumptions=(basicForallExpr, InSet(Lc, S), QLc, RLc))

basicForallExprSpec: {forall_{x in S | Q(x) , R(x)} P(x) , c in S , Q(c) , R(c)} |- P(c)

In [5]:
def testFxn():
    import pdb
    pdb.set_trace()
    return basicForallExpr.usedVars()

In [6]:
basicForallExprSpec.usedVars()

{P}

In [7]:
basicForallExprSpec.usedLiterals()

{c}

In [8]:
(list(basicForallExprSpec.subExprIter()))

[P, c]

In [9]:
list(((list(basicForallExprSpec.subExprIter()))[1]).subExprIter())

[]

In [10]:
Lc.markAsConstrained()

In [11]:
Lc._constrained

True

In [12]:
# notice now that re-marking as constrained does not elicit an error
Lc.markAsConstrained()

In [13]:
# Check if Lc is constrained. Returns False if no _constrained attribute;
# otherwise returns value of the _constrained attribute
Lc.isConstrained()

True

In [14]:
# but we can still delete the _constrained attribute, which is a little weird:
if hasattr(Lc, '_constrained'):
    delattr(Lc, '_constrained')
hasattr(Lc, '_constrained')

False

In [15]:
# now calling isConstrained will return false
# even though the attribute doesn't exist:
Lc.isConstrained()

False

In [16]:
basicThereExistsExpr = Exists(x, Px, conditions=[Qx], domain=S)

basicThereExistsExpr: exists_{x in S | Q(x)} P(x)

In [17]:
basicThereExistsExpr

exists_{x in S | Q(x)} P(x)

In [18]:
basicThereExistsExprThm = basicThereExistsExpr.prove(assumptions={basicThereExistsExpr})

basicThereExistsExprThm: {exists_{x in S | Q(x)} P(x)} |- exists_{x in S | Q(x)} P(x)

In [19]:
basicThereExistsExpr02 = Exists({x}, Equals(Add(x, Lc), zero), conditions=[Qx], domain=S)

basicThereExistsExpr02: exists_{x in S | Q(x)} ((x + c) = 0)

In [20]:
basicThereExistsExpr02Thm = basicThereExistsExpr02.prove(assumptions = {basicThereExistsExpr02} )

basicThereExistsExpr02Thm: {exists_{x in S | Q(x)} ((x + c) = 0)} |- exists_{x in S | Q(x)} ((x + c) = 0)

In [21]:
basicThereExistsExpr03 = Exists({x, y}, Pxy, conditions=[Qx, Ry], domain=S)

basicThereExistsExpr03: exists_{x, y in S | Q(x) , R(y)} P(x , y)

In [22]:
basicThereExistsExpr03.instanceExpr

exists_{y in S | R(y)} P(x , y)

In [23]:
basicThereExistsExpr03Thm = basicThereExistsExpr03.prove(assumptions = {basicThereExistsExpr03} )

basicThereExistsExpr03Thm: {exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- exists_{x, y in S | Q(x) , R(y)} P(x , y)

In [24]:
basicThereExistsExprThm

{exists_{x in S | Q(x)} P(x)} |- exists_{x in S | Q(x)} P(x)

In [25]:
QLc = Function(Q, Lc)
basicThereExistsExprThm.skolemize({x:Lc}, assumptions={QLc, InSet(Lc, S)})

remainingSubVars =  {x}
Things seemed to have worked OK. numExistsEliminations =  1


{Q(c) , c in S , exists_{x in S | Q(x)} P(x)} |- P(c)

In [26]:
basicThereExistsExpr03Thm

{exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- exists_{x, y in S | Q(x) , R(y)} P(x , y)

In [27]:
QLc = Function(Q, Lc)
RLd = Function(R, Ld)
PLcLd = Function(P, {Lc, Ld})
# the following should be problematic if it attempts to re-use the x:Lc skolemization mapping
# because the Lc Literal was previously used in a skolemization above:
# basicThereExistsExpr03Thm.skolemize({x:Lc, y:Ld}, assumptions={InSet(Lc, S), InSet(Ld, S), QLc, RLd, PLcLd})


from proveit import ProofFailure
try:
    basicThereExistsExpr03Thm.skolemize({x:Lc, y:Ld}, assumptions={InSet(Lc, S), InSet(Ld, S), QLc, RLd, PLcLd})
    assert False, "Expecting a Skolemization error; should not make it to this point."
except ProofFailure as e:
    print("EXPECTED ERROR:", e)

remainingSubVars =  {x, y}
EXPECTED ERROR: Proof step failed assuming {R(d), P(d , c), d in S, Q(c), c in S, exists_{x, y in S | Q(x) , R(y)} P(x , y)}: Expecting skolemizeMap substitution values to be Literals not previously used to skolemize. "c" has already been used to skolemize.


In [28]:
Le = Literal('e')
QLe = Function(Q, Le)
RLd = Function(R, Ld)
PLeLd = Function(P, {Le, Ld})
# the following should be problematic if it attempts to re-use the x:Lc skolemization mapping
# because the Lc Literal was previously used in a skolemization above:
basicThereExistsExpr03Thm.skolemize({x:Le, y:Ld}, assumptions={InSet(Le, S), InSet(Ld, S), QLe, RLd, PLeLd})

remainingSubVars =  {x, y}
Things seemed to have worked OK. numExistsEliminations =  2


{R(d) , Q(e) , d in S , e in S , exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- P(e , d)

Attempting to skolemize a Forall expression should fail and produce an error message, alerting the user to the fact that only an Exists expression can be skolemized:

In [29]:
from proveit import ProofFailure
try:
    substitution.skolemize({x:Lc}, assumptions={Qa})
    assert False, "Expecting a Skolemization error; should not make it to this point."
except ProofFailure as e:
    print("EXPECTED ERROR:", e)

remainingSubVars =  {x}
EXPECTED ERROR: Proof step failed assuming {Q(a)}: May only skolemize instance variables of an Exists expression.


But right now, we can still specialize a Forall $\forall$ KnownTruth using a Literal that has been previously used as a Skolem constant:

In [30]:
substitutionSpec = substitution.specialize({x:Lc, y:Ld}, assumptions=[Equals(Lc, Ld)])

substitutionSpec: {c = d} |- f(c) = f(d)

In [31]:
%end skolemization_explorations